In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the datasets
dataset_parameters = pd.read_csv("merged.csv")  # Replace with the actual file name
dataset_stock = pd.read_csv("stock_historical.csv")  # Replace with the actual file name

# Ensure year column is treated consistently
dataset_parameters["Year"] = dataset_parameters["Year"].astype(int)
dataset_stock["Year"] = dataset_stock["Year"].astype(int)

# Prepare data for each country
countries = dataset_parameters["Company_name"].unique()
predictions_per_country = {}

for country in countries:
    # Extract data for the current country
    country_parameters = dataset_parameters[dataset_parameters["Company_name"] == country]
    country_stock = dataset_stock[dataset_stock["Company_name"] == country]

    # Merge the datasets on year
    merged_data = pd.merge(country_parameters, country_stock, on=["Company_name", "Year"], how="inner")

    # Separate features (parameters) and target (repurchase of common stock)
    X = merged_data[["Parameter1", "Parameter2", "Parameter3", "Parameter4", "Parameter5"]].values
    y = merged_data["Stock_Price"].values

    # Scale the input features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Build the neural network model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='linear')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=8, verbose=0)

    # Predict for future years (2024-2033)
    future_years = range(2024, 2034)
    future_predictions = []

    for year in future_years:
        # Extract parameters for the year from the dataset
        future_parameters = country_parameters[country_parameters["Year"] == year]

        if future_parameters.empty:
            print(f"Parameters for {country} in {year} are missing. Skipping prediction.")
            continue

        # Prepare input data
        X_future = future_parameters[["Parameter1", "Parameter2", "Parameter3", "Parameter4", "Parameter5"]].values
        X_future_scaled = scaler.transform(X_future)

        # Predict the stock price
        prediction = model.predict(X_future_scaled)
        future_predictions.append(prediction[0][0])

    # Store predictions for the current country
    predictions_per_country[country] = {year: pred for year, pred in zip(future_years, future_predictions)}

# Display the predictions for each country
for country, predictions in predictions_per_country.items():
    print(f"Predictions for {country}:")
    for year, pred in predictions.items():
        print(f"  Year {year}: {pred:.2f}")

KeyError: 'Year'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the datasets
try:
    dataset_parameters = pd.read_csv("merged.csv")  # Replace with the actual file name
    dataset_stock = pd.read_csv("stock_historical.csv")  # Replace with the actual file name
except FileNotFoundError as e:
    print(f"Error: {e}. Please check the file paths.")
    exit()

# Check for the presence of the 'Year' column
print("Dataset Parameters Columns:", dataset_parameters.columns)
print("Dataset Stock Columns:", dataset_stock.columns)

# Ensure consistent column names for 'Year'
if "Year" not in dataset_parameters.columns:
    print("Year column missing in dataset_parameters! Please verify the CSV file.")
    exit()

if "Year" not in dataset_stock.columns:
    print("Year column missing in dataset_stock! Please verify the CSV file.")
    exit()

# Ensure 'Year' column is of integer type
dataset_parameters["Year"] = dataset_parameters["Year"].astype(int)
dataset_stock["Year"] = dataset_stock["Year"].astype(int)

# Prepare data for each country
countries = dataset_parameters["Company_name"].unique()
predictions_per_country = {}

for country in countries:
    # Extract data for the current country
    country_parameters = dataset_parameters[dataset_parameters["Company_name"] == country]
    country_stock = dataset_stock[dataset_stock["Company_name"] == country]

    # Merge the datasets on year
    merged_data = pd.merge(country_parameters, country_stock, on=["Company_name", "Year"], how="inner")

    # Check if merged data has required columns
    required_columns = ["Parameter1", "Parameter2", "Parameter3", "Parameter4", "Parameter5", "Stock_Price"]
    if not all(col in merged_data.columns for col in required_columns):
        print(f"Missing required columns for {country}. Skipping.")
        continue

    # Separate features (parameters) and target (repurchase of common stock)
    X = merged_data[["Parameter1", "Parameter2", "Parameter3", "Parameter4", "Parameter5"]].values
    y = merged_data["Stock_Price"].values

    # Scale the input features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Build the neural network model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='linear')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=8, verbose=0)

    # Predict for future years (2024-2033)
    future_years = range(2024, 2034)
    future_predictions = []

    for year in future_years:
        # Extract parameters for the year from the dataset
        future_parameters = country_parameters[country_parameters["Year"] == year]

        if future_parameters.empty:
            print(f"Parameters for {country} in {year} are missing. Skipping prediction.")
            continue

        # Prepare input data
        X_future = future_parameters[["Parameter1", "Parameter2", "Parameter3", "Parameter4", "Parameter5"]].values
        X_future_scaled = scaler.transform(X_future)

        # Predict the stock price
        prediction = model.predict(X_future_scaled)
        future_predictions.append(prediction[0][0])

    # Store predictions for the current country
    predictions_per_country[country] = {year: pred for year, pred in zip(future_years, future_predictions)}

# Display the predictions for each country
for country, predictions in predictions_per_country.items():
    print(f"Predictions for {country}:")
    for year, pred in predictions.items():
        print(f"  Year {year}: {pred:.2f}")


Dataset Parameters Columns: Index(['Company_name', 'Parameters', '2019', '2020', '2021', '2022', '2023',
       '2024', '2025', '2026', '2027', '2028', '2029', '2030', '2031', '2032',
       '2033'],
      dtype='object')
Dataset Stock Columns: Index(['Company_name', 'Parameters', '2019', '2020', '2021', '2022', '2023'], dtype='object')
Year column missing in dataset_parameters! Please verify the CSV file.
Year column missing in dataset_stock! Please verify the CSV file.


KeyError: 'Year'

: 